In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

In [4]:
df=pd.read_csv(r"D:\1\Ali\Machine_Learning\Recommendation_System\cleanedfile.csv")
df

,title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language
0,Complaints,pop,Somatina,2019,68,None,[Verse 1]\nI don't want to look like a frustra...,5133768,en,en,en
1,12AM,rap,Lil Macks,2021,6443,None,"[Chorus]\nIt's 12am, I found myself in this ri...",7297758,en,en,en
2,Fantastic,rap,LMEN PRALA,2021,2027,{Curtis},"[Refrén]\nMég táncolj, rázzad, ne parázz bae\n...",7073645,hu,hu,hu
3,VIOLENT 3,rap,UG Planet,2021,575,"{Fairy5100,Ayomagg,LuvLilGeo,Raidurr,""Kidd Aki...","[Intro]\n(Goated, this shit is crazy)\n\n[Vers...",7274717,en,en,en
4,Snow White,pop,Zack Martino,2021,41,"{""Kyle Reynolds""}","[Verse 1]\nShe don't need money, she likes to ...",7188507,en,en,en
...,...,...,...,...,...,...,...,...,...,...,...
190503,Ein Böser Scherz,pop,Ewigheim,2015,88,None,"Mir schmerzt der Kopf, gibt keine Ruh\nNicht d...",1150078,de,de,de
190504,Give It More,rap,Canibus,2006,598,None,"[Verse One]\nI got this beat from Riggs, yeah ...",12828,en,en,en
190505,Picayune,pop,Yori,2018,605,None,[Intro]\nSunday nights I'm all alone\nI ain't ...,5977859,en,en,en
190506,아름다운 날 Beautiful day,pop,LUNA (f(x)),2010,4,None,그리워 져요 그리워 져요\n그대와 함께할 그날 따스했던 그날이\n보고 싶어요 다시 ...,6480670,ko,ko,ko


In [5]:
df.isna().sum()

title            0
tag              0
artist           0
year             0
views            0
features         0
lyrics           0
id               0
language_cld3    0
language_ft      0
language         0
dtype: int64

In [8]:

label_encoder = LabelEncoder()
text_values = np.array(df['artist'])
df['artist'] = label_encoder.fit_transform(text_values)
df

,title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language
0,Complaints,pop,85920,2019,68,None,[Verse 1]\nI don't want to look like a frustra...,5133768,en,en,en
1,12AM,rap,55809,2021,6443,None,"[Chorus]\nIt's 12am, I found myself in this ri...",7297758,en,en,en
2,Fantastic,rap,52996,2021,2027,{Curtis},"[Refrén]\nMég táncolj, rázzad, ne parázz bae\n...",7073645,hu,hu,hu
3,VIOLENT 3,rap,97075,2021,575,"{Fairy5100,Ayomagg,LuvLilGeo,Raidurr,""Kidd Aki...","[Intro]\n(Goated, this shit is crazy)\n\n[Vers...",7274717,en,en,en
4,Snow White,pop,103714,2021,41,"{""Kyle Reynolds""}","[Verse 1]\nShe don't need money, she likes to ...",7188507,en,en,en
...,...,...,...,...,...,...,...,...,...,...,...
190503,Ein Böser Scherz,pop,31121,2015,88,None,"Mir schmerzt der Kopf, gibt keine Ruh\nNicht d...",1150078,de,de,de
190504,Give It More,rap,16683,2006,598,None,"[Verse One]\nI got this beat from Riggs, yeah ...",12828,en,en,en
190505,Picayune,pop,102383,2018,605,None,[Intro]\nSunday nights I'm all alone\nI ain't ...,5977859,en,en,en
190506,아름다운 날 Beautiful day,pop,53150,2010,4,None,그리워 져요 그리워 져요\n그대와 함께할 그날 따스했던 그날이\n보고 싶어요 다시 ...,6480670,ko,ko,ko


In [9]:
columns_to_encode = ['language_cld3', 'language_ft', 'language']

x = df[columns_to_encode]
encoder = MultiLabelBinarizer()
y = encoder.fit_transform(x.values)
encoded_df = pd.DataFrame(y, columns=encoder.classes_)
df2 = pd.concat([df, encoded_df], axis=1)
df2.drop(columns=columns_to_encode, inplace=True)
df2

,title,tag,artist,year,views,features,lyrics,id,af,am,...,sr,sv,ta,te,th,tr,uk,ur,vi,zh
0,Complaints,pop,85920,2019,68,None,[Verse 1]\nI don't want to look like a frustra...,5133768,0,0,...,0,0,0,0,0,0,0,0,0,0
1,12AM,rap,55809,2021,6443,None,"[Chorus]\nIt's 12am, I found myself in this ri...",7297758,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Fantastic,rap,52996,2021,2027,{Curtis},"[Refrén]\nMég táncolj, rázzad, ne parázz bae\n...",7073645,0,0,...,0,0,0,0,0,0,0,0,0,0
3,VIOLENT 3,rap,97075,2021,575,"{Fairy5100,Ayomagg,LuvLilGeo,Raidurr,""Kidd Aki...","[Intro]\n(Goated, this shit is crazy)\n\n[Vers...",7274717,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Snow White,pop,103714,2021,41,"{""Kyle Reynolds""}","[Verse 1]\nShe don't need money, she likes to ...",7188507,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190503,Ein Böser Scherz,pop,31121,2015,88,None,"Mir schmerzt der Kopf, gibt keine Ruh\nNicht d...",1150078,0,0,...,0,0,0,0,0,0,0,0,0,0
190504,Give It More,rap,16683,2006,598,None,"[Verse One]\nI got this beat from Riggs, yeah ...",12828,0,0,...,0,0,0,0,0,0,0,0,0,0
190505,Picayune,pop,102383,2018,605,None,[Intro]\nSunday nights I'm all alone\nI ain't ...,5977859,0,0,...,0,0,0,0,0,0,0,0,0,0
190506,아름다운 날 Beautiful day,pop,53150,2010,4,None,그리워 져요 그리워 져요\n그대와 함께할 그날 따스했던 그날이\n보고 싶어요 다시 ...,6480670,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
encoder = OneHotEncoder(sparse_output=False)
tags = df2['tag'].values.reshape(-1, 1)
x = encoder.fit_transform(tags)
y = pd.DataFrame(x, columns=encoder.get_feature_names_out(['tag']))
df2 = pd.concat([df2, y], axis=1)
df2.drop(columns=['tag'], inplace=True)
df2

,title,artist,year,views,features,lyrics,id,af,am,ar,...,uk,ur,vi,zh,tag_country,tag_misc,tag_pop,tag_rap,tag_rb,tag_rock
0,Complaints,85920,2019,68,None,[Verse 1]\nI don't want to look like a frustra...,5133768,0,0,0,...,0,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0
1,12AM,55809,2021,6443,None,"[Chorus]\nIt's 12am, I found myself in this ri...",7297758,0,0,0,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0
2,Fantastic,52996,2021,2027,{Curtis},"[Refrén]\nMég táncolj, rázzad, ne parázz bae\n...",7073645,0,0,0,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0
3,VIOLENT 3,97075,2021,575,"{Fairy5100,Ayomagg,LuvLilGeo,Raidurr,""Kidd Aki...","[Intro]\n(Goated, this shit is crazy)\n\n[Vers...",7274717,0,0,0,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0
4,Snow White,103714,2021,41,"{""Kyle Reynolds""}","[Verse 1]\nShe don't need money, she likes to ...",7188507,0,0,0,...,0,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190503,Ein Böser Scherz,31121,2015,88,None,"Mir schmerzt der Kopf, gibt keine Ruh\nNicht d...",1150078,0,0,0,...,0,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0
190504,Give It More,16683,2006,598,None,"[Verse One]\nI got this beat from Riggs, yeah ...",12828,0,0,0,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0
190505,Picayune,102383,2018,605,None,[Intro]\nSunday nights I'm all alone\nI ain't ...,5977859,0,0,0,...,0,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0
190506,아름다운 날 Beautiful day,53150,2010,4,None,그리워 져요 그리워 져요\n그대와 함께할 그날 따스했던 그날이\n보고 싶어요 다시 ...,6480670,0,0,0,...,0,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0


In [13]:
df2.to_csv(r"D:\1\Ali\Machine_Learning\Recommendation_System\encoded_cleaned.csv", index=False)

In [14]:
X=pd.read_csv(r"D:\1\Ali\Machine_Learning\Recommendation_System\encoded_cleaned.csv")
X

,title,artist,year,views,features,lyrics,id,af,am,ar,...,uk,ur,vi,zh,tag_country,tag_misc,tag_pop,tag_rap,tag_rb,tag_rock
0,Complaints,85920,2019,68,None,[Verse 1]\nI don't want to look like a frustra...,5133768,0,0,0,...,0,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0
1,12AM,55809,2021,6443,None,"[Chorus]\nIt's 12am, I found myself in this ri...",7297758,0,0,0,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0
2,Fantastic,52996,2021,2027,{Curtis},"[Refrén]\nMég táncolj, rázzad, ne parázz bae\n...",7073645,0,0,0,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0
3,VIOLENT 3,97075,2021,575,"{Fairy5100,Ayomagg,LuvLilGeo,Raidurr,""Kidd Aki...","[Intro]\n(Goated, this shit is crazy)\n\n[Vers...",7274717,0,0,0,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0
4,Snow White,103714,2021,41,"{""Kyle Reynolds""}","[Verse 1]\nShe don't need money, she likes to ...",7188507,0,0,0,...,0,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190503,Ein Böser Scherz,31121,2015,88,None,"Mir schmerzt der Kopf, gibt keine Ruh\nNicht d...",1150078,0,0,0,...,0,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0
190504,Give It More,16683,2006,598,None,"[Verse One]\nI got this beat from Riggs, yeah ...",12828,0,0,0,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0
190505,Picayune,102383,2018,605,None,[Intro]\nSunday nights I'm all alone\nI ain't ...,5977859,0,0,0,...,0,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0
190506,아름다운 날 Beautiful day,53150,2010,4,None,그리워 져요 그리워 져요\n그대와 함께할 그날 따스했던 그날이\n보고 싶어요 다시 ...,6480670,0,0,0,...,0,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0


In [15]:
y=X['views']
X.drop(["title","views","features","lyrics","id"],inplace=True,axis=1)
X

,artist,year,af,am,ar,az,be,bg,bn,ca,...,uk,ur,vi,zh,tag_country,tag_misc,tag_pop,tag_rap,tag_rb,tag_rock
0,85920,2019,0,0,0,0,0,0,0,0,...,0,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0
1,55809,2021,0,0,0,0,0,0,0,0,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0
2,52996,2021,0,0,0,0,0,0,0,0,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0
3,97075,2021,0,0,0,0,0,0,0,0,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0
4,103714,2021,0,0,0,0,0,0,0,0,...,0,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190503,31121,2015,0,0,0,0,0,0,0,0,...,0,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0
190504,16683,2006,0,0,0,0,0,0,0,0,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0
190505,102383,2018,0,0,0,0,0,0,0,0,...,0,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0
190506,53150,2010,0,0,0,0,0,0,0,0,...,0,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0


In [16]:
X.columns.tolist()

['artist',
 'year',
 'af',
 'am',
 'ar',
 'az',
 'be',
 'bg',
 'bn',
 'ca',
 'ceb',
 'cs',
 'cy',
 'da',
 'de',
 'el',
 'en',
 'eo',
 'es',
 'et',
 'eu',
 'fa',
 'fi',
 'fil',
 'fr',
 'fy',
 'ga',
 'gl',
 'he',
 'hi',
 'hr',
 'hu',
 'hy',
 'id.1',
 'is',
 'it',
 'ja',
 'ka',
 'kk',
 'km',
 'ko',
 'ku',
 'ky',
 'la',
 'lt',
 'lv',
 'mg',
 'mk',
 'mn',
 'ms',
 'my',
 'ne',
 'nl',
 'no',
 'pa',
 'pl',
 'pt',
 'ro',
 'ru',
 'si',
 'sk',
 'sl',
 'sq',
 'sr',
 'sv',
 'ta',
 'te',
 'th',
 'tr',
 'uk',
 'ur',
 'vi',
 'zh',
 'tag_country',
 'tag_misc',
 'tag_pop',
 'tag_rap',
 'tag_rb',
 'tag_rock']

In [19]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
import numpy as np
from scipy.stats import randint
sample_size = 100000
valid_indices = X.shape[0]
random_indices = np.random.choice(valid_indices, size=sample_size, replace=False)
X_sample = X.iloc[random_indices] 
y_sample = y.iloc[random_indices] 

In [22]:
X_sample_train, X_sample_test, y_sample_train, y_sample_test = train_test_split(X, y, test_size=0.2, random_state=42)

param_dist = {
    'n_neighbors': randint(1, 10),  # Number of neighbors
    'weights': ['uniform','distance'],
    'p': [1,2]  # 1 for Manhattan distance, 2 for Euclidean distance
}

model = KNeighborsClassifier()

randomized_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    scoring='accuracy',
    cv=3,
    n_iter=5,
)
randomized_search.fit(X_sample_train, y_sample_train)

best_model = randomized_search.best_estimator_
best_params = randomized_search.best_params_
accuracy = best_model.score(X_sample_test, y_sample_test)

print("Best Hyperparameters:", best_params)
print(accuracy)

C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


Best Hyperparameters: {'n_neighbors': 9, 'p': 1, 'weights': 'uniform'}
0.016587055797595927


In [23]:

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10]
}

model = RandomForestRegressor()
randomized_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_grid,
    scoring='neg_mean_squared_error',
    cv=3,
    n_iter=5,
)
randomized_search.fit(X_sample_train, y_sample_train)

best_model = randomized_search.best_estimator_
best_params = randomized_search.best_params_
score = best_model.score(X_sample_test, y_sample_test)

print("Best Hyperparameters:", best_params)
print(score)

Best Hyperparameters: {'n_estimators': 300, 'min_samples_split': 5, 'max_depth': 5}
0.007600829649247465


In [26]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
 

linear_reg = LinearRegression()

# Train the model on the training data
linear_reg.fit(X_train, y_train)

# Predictions on the test set
test_predictions = linear_reg.predict(X_test)

# Calculate RMSE
rmse_test = np.sqrt(mean_squared_error(y_test, test_predictions))

print("Test Set RMSE:", rmse_test)


Test Set RMSE: 29319.59253610312


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from scipy.stats import uniform


# Standardize the data
scaler = StandardScaler()
X_standardized = scaler.fit_transform(X)

# Define the hyperparameters and their distributions
param_dist = {
    'C': uniform(loc=0, scale=1),
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga']
}

# Create an instance of the model
model = LogisticRegression()

# Perform randomized search with cross-validation
randomized_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, scoring='accuracy', cv=5, n_iter=10)
randomized_search.fit(X_standardized, y)

# Get the best model and its hyperparameters
best_model = randomized_search.best_estimator_
best_params = randomized_search.best_params_